In [ ]:
! wget https://docker-76009.sz.gfp.tencent-cloud.com/github/cube-studio/aihub/ml/catboost/adult.data

--2022-10-10 19:55:28--  https://docker-76009.sz.gfp.tencent-cloud.com/github/cube-studio/aihub/ml/catboost/adult.data
Resolving docker-76009.sz.gfp.tencent-cloud.com (docker-76009.sz.gfp.tencent-cloud.com)... 183.47.104.64, 183.47.104.100, 183.47.114.103, ...
Connecting to docker-76009.sz.gfp.tencent-cloud.com (docker-76009.sz.gfp.tencent-cloud.com)|183.47.104.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/octet-stream]
Saving to: ‘adult.data’

adult.data            0%[                    ]       0  --.-KB/s               

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import f1_score

# 读取数据
data = pd.read_csv('./adult.data', header=None)
# 变量重命名
data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
                'marital-status', 'occupation', 'relationship', 'race', 'sex', 
                'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
# 标签转换
data['income'] = data['income'].astype("category").cat.codes
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(data.drop(['income'], axis=1), data['income'],
                                                    random_state=10, test_size=0.3)
# 配置训练参数
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=4, iterations=500, l2_leaf_reg=1,
                            learning_rate=0.1)
# 类别特征索引
cat_features_index = [1, 3, 5, 6, 7, 8, 9, 13]
# 训练
clf.fit(X_train, y_train, cat_features=cat_features_index)
# 预测
y_pred = clf.predict(X_test)
# 测试集f1得分
print(f1_score(y_test, y_pred))

(32561, 15)

In [2]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
data.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
data['income'] = data['income'].astype("category").cat.codes
data['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(data.drop(['income'], axis=1), data['income'],
                                                    random_state=10, test_size=0.3)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(22792, 14) (22792,) (9769, 14) (9769,)


In [9]:
import catboost as cb
cat_features_index = [1, 3, 5, 6, 7, 8, 9, 13]
params = {'depth': [4, 6],
          'learning_rate': [0.01, 0.1],
          'l2_leaf_reg': [1, 4, 9],
          'iterations': [500]}

cb = cb.CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv=3)
cb_model.fit(X_train, y_train, cat_features=cat_features_index)

0:	learn: 0.6835058	total: 61.7ms	remaining: 30.8s
1:	learn: 0.6742830	total: 90.4ms	remaining: 22.5s
2:	learn: 0.6654358	total: 139ms	remaining: 23.1s
3:	learn: 0.6571546	total: 170ms	remaining: 21.1s
4:	learn: 0.6492061	total: 188ms	remaining: 18.6s
5:	learn: 0.6411828	total: 216ms	remaining: 17.8s
6:	learn: 0.6333846	total: 271ms	remaining: 19.1s
7:	learn: 0.6256674	total: 311ms	remaining: 19.1s
8:	learn: 0.6185847	total: 339ms	remaining: 18.5s
9:	learn: 0.6111679	total: 365ms	remaining: 17.9s
10:	learn: 0.6039260	total: 392ms	remaining: 17.4s
11:	learn: 0.5970827	total: 410ms	remaining: 16.7s
12:	learn: 0.5906176	total: 437ms	remaining: 16.4s
13:	learn: 0.5842135	total: 455ms	remaining: 15.8s
14:	learn: 0.5776900	total: 478ms	remaining: 15.5s
15:	learn: 0.5716740	total: 526ms	remaining: 15.9s
16:	learn: 0.5653441	total: 555ms	remaining: 15.8s
17:	learn: 0.5596729	total: 574ms	remaining: 15.4s
18:	learn: 0.5538479	total: 610ms	remaining: 15.4s
19:	learn: 0.5484339	total: 627ms	remai

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001D05BB49308>,
             param_grid={'depth': [4, 6], 'iterations': [500],
                         'l2_leaf_reg': [1, 4, 9],
                         'learning_rate': [0.01, 0.1]},
             scoring='roc_auc')

In [10]:
print(cb_model.best_score_)  
print(cb_model.best_params_)

0.9275055301975431
{'depth': 4, 'iterations': 500, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


In [13]:
import catboost as cb
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=4, iterations=500, l2_leaf_reg=1,
                            learning_rate=0.1)

clf.fit(X_train, y_train, cat_features=cat_features_index)

0:	total: 77.2ms	remaining: 38.5s
1:	total: 132ms	remaining: 32.9s
2:	total: 164ms	remaining: 27.2s
3:	total: 195ms	remaining: 24.2s
4:	total: 228ms	remaining: 22.6s
5:	total: 288ms	remaining: 23.7s
6:	total: 310ms	remaining: 21.8s
7:	total: 353ms	remaining: 21.7s
8:	total: 395ms	remaining: 21.5s
9:	total: 435ms	remaining: 21.3s
10:	total: 477ms	remaining: 21.2s
11:	total: 515ms	remaining: 20.9s
12:	total: 557ms	remaining: 20.9s
13:	total: 587ms	remaining: 20.4s
14:	total: 629ms	remaining: 20.3s
15:	total: 671ms	remaining: 20.3s
16:	total: 720ms	remaining: 20.5s
17:	total: 762ms	remaining: 20.4s
18:	total: 804ms	remaining: 20.3s
19:	total: 855ms	remaining: 20.5s
20:	total: 913ms	remaining: 20.8s
21:	total: 978ms	remaining: 21.3s
22:	total: 1.02s	remaining: 21.2s
23:	total: 1.1s	remaining: 21.9s
24:	total: 1.17s	remaining: 22.1s
25:	total: 1.21s	remaining: 22.1s
26:	total: 1.26s	remaining: 22.1s
27:	total: 1.3s	remaining: 21.9s
28:	total: 1.35s	remaining: 21.9s
29:	total: 1.39s	remainin

In [14]:
y_pred = clf.predict(X_test)
y_pred.shape

(9769,)

In [16]:
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred))

0.7112659698025551


In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8727607738765483

In [18]:
from sklearn.metrics import classification_report
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.90      0.94      0.92      7423\n           1       0.78      0.65      0.71      2346\n\n    accuracy                           0.87      9769\n   macro avg       0.84      0.80      0.81      9769\nweighted avg       0.87      0.87      0.87      9769\n'